<a href="https://colab.research.google.com/github/Pushparaj-Madhu/Pytest250225/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# case_request_db.py

from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

# Setup database (SQLite example, change for PostgreSQL/MySQL if needed)
DATABASE_URL = "sqlite:///E:/DOCS_Placement/PythonWorkSpace/case_request.db"
engine = create_engine(DATABASE_URL, echo=False)

Base = declarative_base()
Session = sessionmaker(bind=engine)


# -----------------------
# Case Table
# -----------------------
class Case(Base):
    __tablename__ = "cases"

    caseid = Column(Integer, primary_key=True, autoincrement=True)
    case_name = Column(String(100))   # Example attribute
    case_status = Column(String(50))  # Example attribute
    # ... add 13 more attributes

    # One-to-many relationship
    requests = relationship("Request", back_populates="case", cascade="all, delete-orphan")


# -----------------------
# Request Table
# -----------------------
class Request(Base):
    __tablename__ = "requests"

    requestid = Column(Integer, primary_key=True, autoincrement=True)
    request_type = Column(String(100))   # Example attribute
    request_status = Column(String(50))  # Example attribute
    # ... add 68 more attributes

    # Foreign key relationship
    caseid = Column(Integer, ForeignKey("cases.caseid"))
    case = relationship("Case", back_populates="requests")


# -----------------------
# Create Tables
# -----------------------
Base.metadata.create_all(engine)


# -----------------------
# CRUD Operations
# -----------------------

def create_case(case_name, case_status):
    session = Session()
    new_case = Case(case_name=case_name, case_status=case_status)
    session.add(new_case)
    session.commit()
    res=new_case.caseid
    session.close()
    return res


def add_request(caseid, request_type, request_status):
    session = Session()
    new_request = Request(caseid=caseid, request_type=request_type, request_status=request_status)
    session.add(new_request)
    session.commit()
    res = new_request.requestid
    session.close()
    return res


def get_case_with_requests(caseid):
    session = Session()
    case = session.query(Case).filter(Case.caseid == caseid).first()
    session.close()
    return case


def update_case(caseid, new_status):
    session = Session()
    case = session.query(Case).filter(Case.caseid == caseid).first()
    if case:
        case.case_status = new_status
        session.commit()
    session.close()


def delete_case(caseid):
    session = Session()
    case = session.query(Case).filter(Case.caseid == caseid).first()
    if case:
        session.delete(case)
        session.commit()
    session.close()


# -----------------------
# Example Usage
# -----------------------
if __name__ == "__main__":
    pass
    # Create Case
    caseid = create_case("Fraud Investigation", "Open")
    print(f"Created Case ID: {caseid}")

    # Add Requests under Case
    add_request(caseid, "Document Verification", "Pending")
    add_request(caseid, "Customer Interview", "In Progress")

    # Fetch Case with Requests
    case = get_case_with_requests(caseid)
    print(f"Case: {case.case_name}, Status: {case.case_status}")

    # for req in case.requests:
    #     print(f"  Request: {req.request_type}, Status: {req.request_status}")

    # Update Case
    # update_case(caseid, "Closed")

    # Delete Case (also deletes requests due to cascade)
    # delete_case(caseid)
